In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from spx_history import *

- date range

In [3]:
start_dt = dt.date(2021,1,1)
end_dt = dt.date.today()

In [6]:
tu = pd.read_csv('data/spy_cst.csv')

In [14]:
tickers = tu[tu['Ticker'] != '-']['Ticker'].unique().tolist()

- split data

In [17]:
def download_data(fun_to_d, tickers, start_dt, end_dt):
	fun_to_name = {
		get_splits : "split",
		get_dividends : "div",
		get_raw_intraday : "intraday"
	}

	# dataset name
	data_name = fun_to_name[fun_to_d]

	# create directory if doesnt exist
	csv_path_dir = f"data/{data_name}/"
	Path(csv_path_dir).mkdir(parents=True, exist_ok=True)

	# failed tickers
	failed_tickers = {}

	for sym in tickers:
		csv_file_path = f"{csv_path_dir}/{sym}.csv"

		if Path(csv_file_path).is_file():
			qt.log.info(f"csv file for {sym} already exists. skip")
			continue
	
		else:
			try:
				qt.log.info(f"querying {data_name} data for symbol {sym}")
				t = fun_to_d(sym, start_dt, end_dt)
				qt.log.info(f"saving {t.shape} rows for symbol {sym} to file {csv_file_path}")
				t.to_csv(csv_file_path, index=False)
			
			except Exception as e:
				qt.log.warning(f"error occurred with ticker : {sym}, error : {e}")
				failed_tickers[sym] = e
	
	return failed_tickers

In [ ]:
download_data(get_dividends, tickers, start_dt, end_dt)

[JUSTY.LOG]	2026-02-05 21:04:25,686 - qt.common.help - INFO - csv file for NVDA already exists. skip
[JUSTY.LOG]	2026-02-05 21:04:25,686 - qt.common.help - INFO - csv file for AAPL already exists. skip
[JUSTY.LOG]	2026-02-05 21:04:25,687 - qt.common.help - INFO - csv file for MSFT already exists. skip
[JUSTY.LOG]	2026-02-05 21:04:25,687 - qt.common.help - INFO - csv file for AMZN already exists. skip
[JUSTY.LOG]	2026-02-05 21:04:25,687 - qt.common.help - INFO - csv file for GOOGL already exists. skip
[JUSTY.LOG]	2026-02-05 21:04:25,688 - qt.common.help - INFO - csv file for GOOG already exists. skip
[JUSTY.LOG]	2026-02-05 21:04:25,688 - qt.common.help - INFO - csv file for META already exists. skip
[JUSTY.LOG]	2026-02-05 21:04:25,688 - qt.common.help - INFO - csv file for AVGO already exists. skip
[JUSTY.LOG]	2026-02-05 21:04:25,689 - qt.common.help - INFO - csv file for TSLA already exists. skip
[JUSTY.LOG]	2026-02-05 21:04:25,689 - qt.common.help - INFO - querying div data for symbol

In [4]:
get_dividends("MSFT", start_date=start_dt, end_date=end_dt)

[JUSTY.LOG]	2026-02-05 19:50:19,715 - qt.common.help - INFO - querying dividend data for symbol MSFT
[JUSTY.LOG]	2026-02-05 19:50:21,489 - qt.common.help - INFO - returned (20, 8) rows


,date,declarationDate,recordDate,paymentDate,period,value,unadjustedValue,currency
0,2021-02-17,2020-12-02,2021-02-18,2021-03-11,Quarterly,0.56,0.56,USD
1,2021-05-19,2021-03-16,2021-05-20,2021-06-10,Quarterly,0.56,0.56,USD
2,2021-08-18,2021-06-16,2021-08-19,2021-09-09,Quarterly,0.56,0.56,USD
3,2021-11-17,2021-09-14,2021-11-18,2021-12-09,Quarterly,0.62,0.62,USD
4,2022-02-16,2021-12-07,2022-02-17,2022-03-10,Quarterly,0.62,0.62,USD
5,2022-05-18,2022-03-14,2022-05-19,2022-06-09,Quarterly,0.62,0.62,USD
6,2022-08-17,2022-06-14,2022-08-18,2022-09-08,Quarterly,0.62,0.62,USD
7,2022-11-16,2022-09-20,2022-11-17,2022-12-08,Quarterly,0.68,0.68,USD
8,2023-02-15,2022-11-29,2023-02-16,2023-03-09,Quarterly,0.68,0.68,USD
9,2023-05-17,2023-03-14,2023-05-18,2023-06-08,Quarterly,0.68,0.68,USD


In [ ]:
get_splits("NFLX", start_date=start_dt, end_date=end_dt)

In [ ]:
ti = get_raw_intraday("NFLX", start_dt, end_dt)